In [1]:
"""
03-08-23 - *Maybe add code to make submission_gt's here, with variable lengths
03-06-23 - Code to split data into train/val.
         - Train/test data was put in main code pipeline for covisit creation
         - For "data/230107_valid.pqt", might've sampled half the sessions and saved
""";

'\n03-08-23 - *Maybe add code to make submission_gt\'s here, with variable lengths\n03-06-23 - Code to split data into train/val.\n         - Train/test data was put in main code pipeline for covisit creation\n         - For "data/230107_valid.pqt", might\'ve sampled half the sessions and saved\n'

In [2]:
import time

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path
import os
import cudf

from otto_utils import *

In [3]:
data = pd.read_parquet('data/230104_df_train.pqt')
data_test = pd.read_parquet('data/230104_df_test.pqt')
d_id2type = pd.read_pickle('data/d_id2type.pkl')
d_type2id = pd.read_pickle('data/d_type2id.pkl')

trange(data)
print(d_id2type, d_type2id)
print(data.shape)
data.head(3)

2022-07-31 22:00:00
2022-08-28 21:59:59
27 days 23:59:59
{0: 'clicks', 1: 'carts', 2: 'orders'} {'clicks': 0, 'carts': 1, 'orders': 2}
(216716096, 4)


,session,ts,type,aid
0,0,1659304800,0,1517085
1,0,1659304904,0,1563459
2,0,1659367439,0,1309446


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216716096 entries, 0 to 216716095
Data columns (total 4 columns):
 #   Column   Dtype
---  ------   -----
 0   session  int64
 1   ts       int32
 2   type     uint8
 3   aid      int32
dtypes: int32(2), int64(1), uint8(1)
memory usage: 3.4 GB


In [5]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6928123 entries, 0 to 6928122
Data columns (total 4 columns):
 #   Column   Dtype
---  ------   -----
 0   session  int64
 1   ts       int32
 2   type     uint8
 3   aid      int32
dtypes: int32(2), int64(1), uint8(1)
memory usage: 112.3 MB


In [4]:
%%time
DO_LOCAL_VALIDATION = True

# Split into train validation with last week as validation
time_range = data.ts.max()-data.ts.min() # ts is in secs
time_range_days = time_range/3600/24
print(f'time_range days: {time_range_days:0.1f}')

# Use last week as hold out. Get tsplit to split train and val data
one_week = 3600*24*7
tsplit = data.ts.max()-one_week
print(f'1 wk tsplit: {tsplit}')

if DO_LOCAL_VALIDATION:
    print('Train wks 1-3, Val wk 4')
    tsplit = data.ts.max()-one_week
    df_train = data[data.ts<=tsplit] # changed from < to <= (Radek)
    df_val = data[data.ts>tsplit]

    s1 = set(df_train.session.values)
    s2 = set(df_val.session.values)
    intersect = s1&s2
    print(f'Intersect length: {len(intersect)}')

    # 1-6-23 any train sessions overlap into val set, are removed from val set
    df_val_clean = df_val[~df_val.session.isin(intersect)]
    
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True) # Not used after this code
    df_val_clean = df_val_clean.reset_index(drop=True)
        
    tstart = data.ts.max()-3*one_week
    df_train_test = data[data.ts>tstart].reset_index(drop=True)

time_range days: 28.0
1 wk tsplit: 1661119199
Train wks 1-3, Val wk 4
Intersect length: 3521833
CPU times: user 28.5 s, sys: 3.91 s, total: 32.4 s
Wall time: 30.8 s


In [8]:
trange(df_train)
trange(df_val)
trange(df_train_test)
trange(data_test)
print(df_train.shape, df_val.shape, df_val_clean.shape, df_train_test.shape, data_test.shape)

2022-07-31 22:00:00
2022-08-21 21:59:59
20 days 23:59:59
2022-08-21 22:00:00
2022-08-28 21:59:59
6 days 23:59:59
2022-08-07 22:00:00
2022-08-28 21:59:59
20 days 23:59:59
2022-08-28 22:00:00
2022-09-04 21:59:51
6 days 23:59:51
(163955181, 4) (52760915, 4) (15384577, 4) (163555218, 4) (6928123, 4)


In [9]:
# Use week 4 for validation. Uniform random cutoff for val data and val labels
l_session = []
new_val, new_val_labels = [], []
t1 = time.perf_counter()
for session, grp in df_val_clean.groupby('session'):
    cutoff = np.random.randint(1, len(grp)) # want >=1 item for each val session
    new_val.append(grp.iloc[:cutoff])
    new_val_labels.append(grp.iloc[cutoff:])
    l_session.append([session, cutoff])

new_val = pd.concat(new_val).reset_index(drop=True)
new_val_labels = pd.concat(new_val_labels).reset_index(drop=True)

t2 = time.perf_counter()
print(f'Elasped time: {(t2-t1)/60.:0.1f} m')

Elasped time: 11.3 m


In [22]:
%%time
# Check all test items are also in the train set
# Covisit matrix can add aids that are missing in the the updated train
# For HP tuning, train wk 1-3, validate/predict on 4
# For final test set, train wk 2-4 (keeps distribution?), predict on 5
# **Need to make a final train set for wks 2-4

aids_data = set(data.aid)
aids_test = set(data_test.aid)
aids_df_train = set(df_train.aid)
aids_df_train_test = set(df_train_test.aid)
aids_new_val = set(new_val.aid)
aids_new_val_labels = set(new_val_labels.aid)
aids_val_wk4 = set(df_val_clean.aid)

print(f'# aids in test, but not in train: {len(aids_test - aids_data)}')
print(f'# aids in test, but not in train wk2-4: {len(aids_test - aids_df_train_test)}')
print(f'100 * test-aids not in train wk2-4 / total test-aids: {len(aids_test - aids_df_train_test)/len(aids_test)*100:0.2f}')
print(f'# aids in wk4 val, but not in train wk1-3: {len(aids_val_wk4 - aids_df_train)}')
print(f'# aids in wk4 val-labels, but not in train wk1-3 {len(aids_new_val_labels - aids_df_train)}')
print('***********')

# aids in test, but not in train: 0
# aids in test, but not in train wk2-4: 739
test-aids not in train wk2-4 / total # test-aids: 0.09
# aids in wk4 val, but not in train wk1-3: 24318
# aids in wk4 val-labels, but not in train wk1-3 20194
CPU times: user 1min 57s, sys: 461 ms, total: 1min 57s
Wall time: 1min 56s


In [17]:
valid = new_val_labels.copy()
if 'int' in str(valid.type.dtype):
    valid.type = valid.type.map(lambda idx: d_id2type[idx])
ground_truth = valid.groupby(['session', 'type'])['aid'].apply(list)
ground_truth = ground_truth.reset_index().rename(columns={'aid': 'labels'})
ground_truth.loc[ground_truth.type == 'clicks', 'labels'] = ground_truth.loc[ground_truth.type == 'clicks', 'labels'].str[:1]
ground_truth

,session,type,labels
0,11098528,clicks,[796572]
1,11098528,orders,"[1462506, 950341, 1561739, 92401]"
2,11098529,clicks,[1298277]
3,11098530,carts,[409236]
4,11098530,clicks,[409236]
...,...,...,...
2213334,12899774,clicks,[1399483]
2213335,12899775,clicks,[1760714]
2213336,12899776,clicks,[1737908]
2213337,12899777,clicks,[384045]


In [23]:
# Save all for reproducibility
pd.to_pickle(l_session, 'data_new/230313_val_cutoff_idxs.pkl', protocol=4)
df_train.to_parquet('data_new/230313_train_1to3.pqt')
df_train_test.to_parquet('data_new/230313_train_2to4.pqt')
new_val.to_parquet('data_new/230313_val.pqt')
new_val_labels.to_parquet('data_new/230313_val_labels.pqt')
pd.to_pickle(ground_truth, 'data_new/230313_new_val_labels_gt.pkl', protocol=4)